In [ ]:
from modfs.data.result_load import load_results, add_derived_columns
from pathlib import Path
from typing import cast
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import pandas as pd
import modfs.utils.plotting as plot


ROOT_PATH = Path("../../").resolve()
FIGS_DIR = ROOT_PATH / "data/figs/paper/distributed_scheduling"
FIGS_DIR.mkdir(exist_ok=True, parents=True)

results_path = ROOT_PATH / "data/run"

df_original = load_results(results_path)
df_original_p = add_derived_columns(
    df_original,
    groups={
        "homogeneous": {
            f"printer_cases/duplex/{v}{a}"
            for v in ["bookletA", "bookletB", "bookletAB", "length"]
            for a in ["Uniform", ""]
        },
        "heterogeneous": {
            f"mixed/duplex/{v}{a}"
            for v in ["bookletA", "bookletB", "bookletAB", "length"]
            for a in ["Uniform", ""]
        },
        "deadlines": {"deadlines/bookletAB"},
    },
    gen_subpath="data/gen",
    exclude_paths=[],
)

df_all = df_original_p[(df_original_p["group"] != "deadlines")].copy()
df_all.replace(
    {"modular_algorithm": {"cocktail": "MAS-C", "constraint": "CP", "broadcast": "MAS-B"}},
    inplace=True,
)
figs = {}

In [ ]:
colors = {
    "notimeout": "tab:blue",
    "CP": "tab:green",
    "MAS": "tab:olive",
    "MAS-C": "tab:olive",
    "MAS-B": "tab:orange",
}
markers = {"CP": ".", "MAS": "*", "MAS-C": "*", "MAS-B": "o"}

In [ ]:
# Draw boxplot comparing the makespan
df_compared = plot.baseline_compare(df_all, "CP", "makespan")
display(df_compared.groupby(["group", "modular_algorithm"])["makespan"].mean())

plt.figure(figsize=(2.2, 2.3), dpi=250)
sns.set_theme(context="paper", style="whitegrid")
g = sns.boxplot(
    data=df_compared,
    x="group",
    y="makespan",
    hue="modular_algorithm",
    hue_order=["MAS-C", "MAS-B"],
    palette=colors,
    legend=False,
    whis=(5, 95),
    showfliers=False,
)

hatches = ["////", "////", "\\\\\\\\", "\\\\\\\\"]

for patch, hatch in zip(g.patches, hatches):
    patch.set_hatch(hatch)

plt.axhline(y=1, color="r")
plt.text(.9, 0.93, "Baseline")

from matplotlib.patches import Patch
plt.legend(
    handles=[
        Patch(facecolor=colors["MAS-C"], edgecolor="k", hatch="////", label="MAS-C"),
        Patch(facecolor=colors["MAS-B"], edgecolor="k", hatch="\\\\\\\\", label="MAS-B"),
    ],
    loc="upper right",
)

plt.ylabel("Relative makespan")
plt.xlabel("Benchmark")
plt.xticks([0, 1], ["Heterogeneous", "Homogeneous"])

plt.savefig(FIGS_DIR / "makespan_all.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
# Draw bar plot comparing number of finished solutions
df_completed = df_all.groupby(["group", "modular_algorithm"])["solved"].mean()
display(df_completed)

plt.figure(figsize=(2.2, 2.3), dpi=250)
sns.set(context="paper", style="whitegrid")

xposition = [0, 0.4, 0.80, 1.3, 1.7, 2.1]
yposition = [
    df_completed["heterogeneous", "MAS-C"],
    df_completed["heterogeneous", "MAS-B"],
    df_completed["heterogeneous", "CP"],
    df_completed["homogeneous", "MAS-C"],
    df_completed["homogeneous", "MAS-B"],
    df_completed["homogeneous", "CP"],
]

plt.bar(
    xposition,
    yposition,
    width=0.4,
    edgecolor="k",
    color=[colors["MAS-C"], colors["MAS-B"], colors["CP"]] * 2,
    hatch=["////", "\\\\\\\\", ""] * 2,
)
from matplotlib.patches import Patch

for x, y in zip(xposition, yposition):
    plt.text(x, y + 0.005, f"{y:.2f}", fontsize="small", horizontalalignment="center")

plt.legend(
    handles=[
        Patch(facecolor=colors["MAS-C"], edgecolor="k", hatch="////", label="MAS-C"),
        Patch(facecolor=colors["MAS-B"], edgecolor="k", hatch="\\\\\\\\", label="MAS-B"),
        Patch(facecolor=colors["CP"], edgecolor="k", label="CP"),
    ],
    loc="lower right",
)
plt.xticks([0.25, 1.75], ["Heterogeneous", "Homogeneous"])
plt.ylabel("Schedulability ratio")
plt.xlabel("Benchmark")
plt.grid(axis="x")
plt.savefig(FIGS_DIR / "completed.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
df_tt = df_all.groupby(["group", "modular_algorithm"])["total_time"].mean()/1000
display(df_tt)

plt.figure(figsize=(2.2, 2.3), dpi=250)
sns.set(context="paper", style="whitegrid")

xposition = [0, 0.4, 0.80, 1.3, 1.7, 2.1]
yposition = [
    df_tt["heterogeneous", "MAS-C"],
    df_tt["heterogeneous", "MAS-B"],
    df_tt["heterogeneous", "CP"],
    df_tt["homogeneous", "MAS-C"],
    df_tt["homogeneous", "MAS-B"],
    df_tt["homogeneous", "CP"],
]

plt.bar(
    xposition,
    yposition,
    width=0.4,
    edgecolor="k",
    color=[colors["MAS-C"], colors["MAS-B"], colors["CP"]] * 2,
    hatch=["////", "\\\\\\\\", ""]*2,
)

plt.legend(handles=[
    Patch(facecolor=colors["MAS-C"], edgecolor="k", hatch="////", label="MAS-C"),
    Patch(facecolor=colors["MAS-B"], edgecolor="k", hatch="\\\\\\\\", label="MAS-B"),
    Patch(facecolor=colors["CP"], edgecolor="k", label="CP"),
], loc="right")

for x, y in zip(xposition, yposition):
    plt.text(x, y + 0.005, f"{y:.0f}", fontsize="small", horizontalalignment="center")


plt.xticks([0.25, 1.75], ["Heterogeneous", "Homogeneous"])
plt.ylabel("Execution time [s]")
plt.xlabel("Benchmark")
plt.grid(axis="x")
plt.savefig(FIGS_DIR / "execution_time_all.pdf", bbox_inches="tight", pad_inches=0)
plt.show()


In [ ]:
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle

df_compared = plot.baseline_compare(df_all, "CP", "makespan")

number_colors = {
    2: "tab:blue",
    3: "tab:orange",
    4: "tab:green",
    5: "tab:red",
    6: "tab:purple",
    7: "tab:brown",
    8: "tab:pink",
    9: "tab:gray",
    10: "tab:olive",
}

number_markers = {
    2: ".",
    3: "1",
    4: "2",
    5: "3",
    6: "4",
    7: "*",
    8: "+",
    9: "x",
    10: 8,
}

plt.figure(figsize=(4, 2.1), dpi=250)
sns.set(context="paper", style="whitegrid")

plt.axhline(1, color="r")
plt.text(90, .9, "Baseline")

for group, df in df_compared[df_compared["modular_algorithm"] == "MAS-C"].groupby("modules"):
    group = cast(int, group)
    # if group % 2 != 0:
    #     continue
    
    df_tmp = df.groupby("jobs", as_index=False)["makespan"].mean()
    plt.plot(
        df_tmp["jobs"],
        df_tmp["makespan"],
        color=number_colors[group],
        marker=number_markers[group],
    ),

l = plt.legend(
    [
        Line2D(
            [],
            [],
            color=number_colors[i],
            marker=number_markers[i],
            markersize=5.0,
        )
        for i in range(2, 11, 1)
    ],
    [f"{i}" for i in range(2, 11, 1)],
    loc="upper left",
    bbox_to_anchor=(-0.02, 1.3),
    title="Modules",
    ncol=5,
    handlelength=1.85,
    columnspacing=.8,
    handletextpad=.6,
)

# From https://stackoverflow.com/a/53329898/4005637
c = l.get_children()[0]
title = c.get_children()[0]
hpack = c.get_children()[1]
c._children = [hpack]
hpack._children = [title] + hpack.get_children()

plt.xlim(0, 305)
plt.ylim(0.6, 1.45)
plt.ylabel("Relative makespan")
plt.xlabel("Number of jobs")
plt.savefig(FIGS_DIR / "makespan_by_jobs_modules.pdf", bbox_inches="tight", pad_inches=0)
plt.show()


In [ ]:
from matplotlib.text import Text

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 2.1), dpi=250, sharey=True)
plt.subplots_adjust(wspace=0.05)
ax2.axhline(10 * 60, color="r")
for group, df in df_all.groupby("modules"):
    df_tmp = df.groupby(["jobs", "modular_algorithm"], as_index=False)["total_time"].mean()
    df_mod = df_tmp[df_tmp["modular_algorithm"] == "MAS-C"]
    group = cast(int, group)
    ax1.plot(
        df_mod["jobs"],
        df_mod["total_time"]/1000,
        color=number_colors[group],
        marker=number_markers[group],
    )
    df_mon = df_tmp[df_tmp["modular_algorithm"] == "CP"]
    ax2.plot(
        df_mon["jobs"],
        df_mon["total_time"]/1000,
        color=number_colors[group],
        marker=number_markers[group],
    )

ax2.text(-85, 10 * 60 - 10, "Time limit")
ax1.set_xlabel("Number of jobs [MAS]")
ax2.set_xlabel("Number of jobs [CP]")
ax1.set_ylabel("Execution time [s]")
plt.ylim(0, 670)
# plt.yscale("log")
l = fig.legend(
    # [ Line2D([], [], color='none') ] + [
    [
        Line2D([0], [0], color=number_colors[i], marker=number_markers[i], markersize=5.0)
        for i in range(2, 11)
    ],
    [f'{i}' for i in range(2, 11)],
    # ["Modules"] + [f'{i}' for i in range(2, 11)],
    loc='upper left',
    bbox_to_anchor=(0.116, 1.03),
    title="Modules",
    ncol=9,
    columnspacing=0.5,
)

# From https://stackoverflow.com/a/53329898/4005637
c = l.get_children()[0]
title = c.get_children()[0]
hpack = c.get_children()[1]
c._children = [hpack]
hpack._children = [title] + hpack.get_children()

# l.get_title().set_position((-630, -45))
plt.savefig(FIGS_DIR / "execution_time_by_jobs_modules.pdf", bbox_inches="tight", pad_inches=0)
plt.show()


In [ ]:
plt.figure(figsize=(3, 2.1), dpi=250)
plt.subplots_adjust(wspace=0.05)

df_tmp = df_all.groupby(["modular_algorithm", "modules"])["total_time"].mean()
plt.plot(
    df_tmp.loc["MAS-C"].index,
    df_tmp.loc["MAS-C"]/1000,
    color=colors["MAS-C"],
    marker=markers["MAS-C"],
    label="MAS-C"
)
plt.plot(
    df_tmp.loc["MAS-B"].index,
    df_tmp.loc["MAS-B"]/1000,
    color=colors["MAS-B"],
    marker=markers["MAS-B"],
    label="MAS-B"
)
plt.plot(
    df_tmp.loc["CP"].index,
    df_tmp.loc["CP"]/1000,
    color=colors["CP"],
    marker=markers["CP"],
    label="CP",
)

plt.axhline(10 * 60, color="r")
plt.text(8, 10 * 60 - 40, "Time limit")
plt.xlabel("Number of modules")
plt.ylabel("Execution time [s]")
plt.ylim(0, 620)
plt.legend(loc=(0.63, 0.4))
plt.savefig(FIGS_DIR / "execution_time_by_modules.pdf", bbox_inches="tight", pad_inches=0)
plt.show()

In [ ]:
plt.figure(figsize=(8, 2.5), dpi=250)
for group, df in df_all.groupby("modules"):
    group = cast(int, group)
    df_tmp = df.groupby(["modular_algorithm", "jobs"])["solved"].mean()
    plt.plot(
        df_tmp.loc["MAS-C"].index,
        df_tmp.loc["MAS-C"],
        color=number_colors[group],
        marker=number_markers[group],
    )
    plt.plot(
        df_tmp.loc["CP"].index,
        df_tmp["CP"],
        color=number_colors[group],
        marker=number_markers[group],
        linestyle="--",
    )
    
plt.ylim(.2, 1.05)
plt.xlim(0, 305)
# plt.xscale("log")
legend = plt.legend(
    [Line2D([], [], color="k"), Line2D([], [], color="k", linestyle="--")],
    ["MAS", "CP"],
)
l = plt.legend(
    [
        Line2D([0], [0], color=number_colors[i], marker=number_markers[i], markersize=5.0)
        for i in range(2, 11)
    ],
    [f"{i}" for i in range(2, 11)],
    loc='upper left',
    bbox_to_anchor=(-.01, 1.17),
    title="Modules",
    ncol=9,
    columnspacing=1.2,
    handletextpad=.8,
    handlelength=2
)

# From https://stackoverflow.com/a/53329898/4005637
c = l.get_children()[0]
title = c.get_children()[0]
hpack = c.get_children()[1]
c._children = [hpack]
hpack._children = [title] + hpack.get_children()

plt.gca().add_artist(legend)
plt.ylabel("Schedulability ratio")
plt.xlabel("Number of jobs")
plt.savefig(FIGS_DIR / "schedulability_ratio.pdf", bbox_inches="tight", pad_inches=0)
plt.show()


In [ ]:
plt.figure(figsize=(3, 2.1), dpi=250)
plt.subplots_adjust(wspace=0.05)

df_tmp = df_all.groupby(["modular_algorithm", "modules"])["makespan"].mean()
plt.plot(
    df_tmp.loc["MAS-C"].index,
    df_tmp.loc["MAS-C"]/1000,
    color=colors["MAS-C"],
    marker=markers["MAS-C"],
    label="MAS-C"
)
plt.plot(
    df_tmp.loc["MAS-B"].index,
    df_tmp.loc["MAS-B"]/1000,
    color=colors["MAS-B"],
    marker=markers["MAS-B"],
    label="MAS-B"
)
plt.plot(
    df_tmp.loc["CP"].index,
    df_tmp.loc["CP"]/1000,
    color=colors["CP"],
    marker=markers["CP"],
    label="CP",
)

plt.axhline(10 * 60, color="r")
plt.text(8, 10 * 60 - 40, "Time limit")
plt.xlabel("Number of modules")
plt.ylabel("Makespan (lower is better)")
# plt.ylim(0, 620)
plt.legend(loc=(0.63, 0.4))
plt.savefig(FIGS_DIR / "makespan_by_modules_all.pdf", bbox_inches="tight", pad_inches=0)
plt.show()